In [1]:
import os
import requests
import json
import sys
import time
import pathlib  # Added pathlib
from requests.auth import HTTPBasicAuth

In [2]:
def read_geom(file_path):
    with open(file_path) as f:
        data = json.load(f)
        if 'features' in data and data['features']:
            bbox = data['features'][0]['geometry']
            return bbox
        else:
            print(f"No features found in {file_path}. Skipping this file.")
            return None
    
def place_order(request, auth):
    response = requests.post('https://api.planet.com/compute/ops/orders/v2', data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    try:
        order_id = response.json()['id']
        print(order_id)
        order_url = 'https://api.planet.com/compute/ops/orders/v2' + '/' + order_id
        return order_url
    except KeyError:
        print(f"Failed to place order for {request['name']}. ID not found.")
        return None

def poll_for_success(order_url, auth):
    while True:
        r = requests.get(order_url, auth=(api_key, ""))
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)

def download_results(results, geojson_file, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    # Extract the parking lot number from the GeoJSON file name
    parking_lot_number = os.path.splitext(geojson_file)[0]
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/Tree_Nation_Polygons/tree_nation_small_size_polygons/small_sizepolys', parking_lot_number, name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            path.parent.mkdir(parents=True, exist_ok=True)  # Create the directory if it doesn't exist
            r = requests.get(url, allow_redirects=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

geojson_folder = '/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/FirstTreePlanted_new'
geojson_files = [f for f in os.listdir(geojson_folder) if f.endswith('.geojson')]

In [3]:
API_keys = []

In [8]:
import os
import sys
import requests
import json
from requests.auth import HTTPBasicAuth
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

try:
    os.environ['PL_API_KEY'] = 'PLAK731e474fdd584ac2a0ecb22def40e20d'
    api_key = os.getenv('PL_API_KEY')
except Exception as e:
    print("Failed")
    sys.exit()

headers = {'Content-Type':'application/json'}
geojson_folder = '/home/idisc02/Saarland_Forest_monitoring_research/Reforestation_Monitoring/Tree_Nation_Polygons/tree_nation_small_size_polygons/small_sizepolys'


geojson_files = [f for f in os.listdir(geojson_folder) if f.endswith('.geojson')]


for geojson_file in geojson_files:
    print(f"Processing {geojson_file}...")
    
    geojson_path = os.path.join(geojson_folder, geojson_file)
    
    clip_aoi = read_geom(geojson_path)
    if clip_aoi is None:
        continue

 

  
    clip = {
        "clip": {
            "aoi": clip_aoi
        }
    }


    geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": clip_aoi
    }

    # Read the GeoJSON file
    with open(geojson_path, 'r') as f:
        data = json.load(f)

    # Check if 'features' exists in the data dictionary and it's not empty
    if 'features' in data and data['features']:
        # Iterate over each feature in the features list
        for feature in data['features']:
            # Check if 'properties' exists in the feature dictionary and 'created_at' is not None
            if 'properties' in feature and feature['properties'].get('created_at'):
                created_at = parse(feature['properties']['created_at'])
                gte_date = created_at + relativedelta(years=5)
                lte_date = gte_date + relativedelta(months=1)
                # Use these dates in your date_range_filter
                date_range_filter = {
                    "type": "DateRangeFilter",
                    "field_name": "acquired",
                    "config": {
                        "gte": gte_date.isoformat(),
                        "lte": lte_date.isoformat()
                        }
                }
            else:
                print(f"'created_at' not found or is None in feature. Skipping this feature.")
                continue

            # Only get images which have <5% cloud coverage
            cloud_cover_filter = {
            "type": "RangeFilter",
            "field_name": "cloud_cover",
            "config": {
                "lt": 0.05
            }
            }

            # Combine our geometry, date, cloud filters
            combined_filter = {
            "type": "AndFilter",
            "config": [geometry_filter, date_range_filter, cloud_cover_filter]
            }

            item_type = "PSScene"
            # API request object
            search_request = {
            "item_types": [item_type], 
            "filter": combined_filter
            }

            # Fire off the POST request for searching imagery
            search_result = requests.post(
                'https://api.planet.com/data/v1/quick-search',
                auth=HTTPBasicAuth(api_key, ''),
                json=search_request)
            
            # Check the status of the response
            if search_result.status_code != 200:
                print(f"Request failed with status code {search_result.status_code}. Skipping this feature.")
                print(search_result.content)
                continue

            # Parse the search result JSON
            geojson = search_result.json()

            # Check if 'features' exists in the geojson dictionary and it's not empty
            if 'features' in geojson and geojson['features']:
                # Extract image IDs from the search results
                image_ids = [feature['id'] for feature in geojson['features']]
            else:
                print(f"No images found that match the filters. Skipping this feature.")
                continue

            # Create an order request with the clip tool and image IDs
            request = {
                "name": os.path.splitext(geojson_file)[0],  # Use the GeoJSON file name without extension
                "products": [
                    {
                        "item_ids": image_ids,
                        "item_type": "PSScene",
                        "product_bundle": "visual",
                    }
                ],
                "tools": [clip]
            }
            print(request)

            order_url = place_order(request, auth=(api_key, ""))
            if order_url is not None:
                print(order_url)
                poll_for_success(order_url, api_key)
                r = requests.get(order_url, auth=(api_key, ""))
                response = r.json()
                results = response['_links']['results']
                print(results)                                                    

                download_results(results, geojson_file)

                print(f"Processing of {geojson_file} complete.")
            else:
                print(f"Skipping download for {geojson_file} due to failed order placement.")

            #clip polygon from image use rasterio
    else:
        print(f"No features found in {geojson_file}. Skipping this file.")

print("Processing of all GeoJSON files complete.")

Processing 393.geojson...
Request failed with status code 400. Skipping this feature.
b'{"field": {"filter.config.1.config.gte": [{"message": "\'2024-08-05T14:45:36ZZ\' is not a \'date-time\'"}], "filter.config.1.config.lte": [{"message": "\'2024-09-05T14:45:36ZZ\' is not a \'date-time\'"}]}, "general": []}'
Processing 841.geojson...
Request failed with status code 400. Skipping this feature.
b'{"field": {"filter.config.1.config.gte": [{"message": "\'2027-01-05T16:29:45ZZ\' is not a \'date-time\'"}], "filter.config.1.config.lte": [{"message": "\'2027-02-05T16:29:45ZZ\' is not a \'date-time\'"}]}, "general": []}'
Processing 60.geojson...
Request failed with status code 400. Skipping this feature.
b'{"field": {"filter.config.1.config.gte": [{"message": "\'2021-07-18T13:41:01ZZ\' is not a \'date-time\'"}], "filter.config.1.config.lte": [{"message": "\'2021-08-18T13:41:01ZZ\' is not a \'date-time\'"}]}, "general": []}'
Processing 20.geojson...
Request failed with status code 400. Skipping 